In [1]:
import pandas as pd
import numpy as np
import random
import csv

In [2]:
df= pd.read_csv('training_data.csv', low_memory=False)

In [3]:
df.shape

(424431, 69)

In [4]:
no_claim = []
claimed = []
for i, x in enumerate(df['Claim_Count']):
    if (int(x) > 0):
        claimed.append(i)
    else:
        no_claim.append(i)
print(len(no_claim))
print(len(claimed))

405562
18869


In [5]:
def drop_unknown(df):
    return df.dropna()

def label_encode(df):
    le = preprocessing.LabelEncoder()
    for col in df.columns:
        col = str(col)
        if str(df.loc[:,col].dtype) == 'object':
            le.fit(df.loc[:,col]) 
            df.loc[:,col] = le.transform(df.loc[:,col])
    return df

def clean_data(df, missing_handler=drop_unknown, data_encoder=label_encode):
    df = missing_handler(df)
    return data_encoder(df)

def standardize(col):
    return col

def pca(df):
    #// TODO: implement PCA
    pass

def preprocess_data(df, **params):
    """
    Preprocesses dataframe, with customizable options.
    
    params:
        clean[Boolean]: should the data be cleaned.
        
        missing_handler[function(dataframe) returns dataframe]: how to handle missing data,
                       'dropna' by default.
                                             
        data_encoder[function(dataframe) returns dataframe]: specifies encoder for data, 
                    'label encoding' is the default.
        
        feature_transform[Boolean]: does feature transformation need to be performed.
        
        feature_transformer[function(column) returns column]: specifies the feature transformer,
                        'Standardization' is the default.
                                          
        feature_selection[Boolean]: does feature selection need to be performed.
        
        feature_selector[function(dataframe) returns dataframe]: specifies the feature selector,
                        'PCA' is the default.
        
    NOTE: all boolean params are set to 'False' by default.
          So the call 'preprocessor()' does nothing, because no arguments are passed.
    """
    do_clean = params.get('clean', False)
    if do_clean:
        missing_handler = params.get('missing_handler', drop_unknown)
        data_encoder = params.get('data_encoder', label_encode)
        df = clean_data(df, missing_handler, data_encoder)
    
    do_transform = params.get('feature_transform', False)
    if do_transform:
        transformer = params.get('feature_transformer', standardize)
        #// TODO: per column transform
        
    do_feature_selection = params.get('feature_selection', False)
    if do_feature_selection:
        feature_selector = params.get('feature_selector', pca)
        df = feature_selector(df)

In [6]:
df.head()

,PolicyNo,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Policy_Zip_Code_Garaging_Location,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,...,EEA_Prior_Bodily_Injury_Limit,EEA_PolicyYear,SYS_Renewed,SYS_New_Business,Annual_Premium,Claim_Count,Loss_Amount,Frequency,Severity,Loss_Ratio
0,164532941,Standard,6,Direct Billed to Insured,Pre-paid,N,43046,35,2004,BUIK LESABRE LI,...,100-400,2006,Y,N,320.12,0,0.0,0.0,0.0,0.0
1,164533241,Standard,6,Direct Billed to Insured,Pre-paid,N,Unknown,35,1980,CADILLAC 4-DOOR,...,100-200,2006,Y,N,259.70,0,0.0,0.0,0.0,0.0
2,164534633,Standard,6,Direct Billed to Insured,Pre-paid,N,43555,17,2005,PONT MONTANA SV,...,100-400,2006,Y,N,613.74,0,0.0,0.0,0.0,0.0
3,164534839,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0
4,164534840,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0


In [7]:
def portfolio_creation(df):
    dic=df.to_dict('records')
    new_dic={}
    for i in range(len(claimed)):
        if dic[claimed[i]]['Claim_Count']>0:
            new_dic[claimed[i]]=dic[claimed[i]]
    noclaim_dic={}
    for i in range(len(no_claim)):
        if dic[no_claim[i]]['Claim_Count']==0:
            noclaim_dic[no_claim[i]]=dic[no_claim[i]]
    print('--------------Dictionaries done--------------------')
    claimsub=[]
    noclaimsub=[]
    row1=1
    while row1<421:
        if row1==420:
            claimsub.append(new_dic)
            break
        csubset = dict(random.sample(new_dic.items(), 45))
        claimsub.append(csubset)
        row1+=1
        for k in csubset.keys():
            del new_dic[k]
    print('--------------Claims done--------------------')
    row2=1
    while row2 < 421:
        if row2==420:
            noclaimsub.append(noclaim_dic)
            break
        nsubset = dict(random.sample(noclaim_dic.items(), 965))
        noclaimsub.append(nsubset)
        row2+=1
        for k in nsubset.keys():
            del noclaim_dic[k]
    print('--------------No claims done--------------------')
    for i in range(420):
        claimsub[i].update(noclaimsub[i])
    csv_columns=['PolicyNo','Policy_Company','Policy_Installment_Term','Policy_Billing_Code','Policy_Method_Of_Payment','Policy_Reinstatement_Fee_Indicator','Policy_Zip_Code_Garaging_Location','Vehicle_Territory','Vehicle_Make_Year','Vehicle_Make_Description','Vehicle_Performance','Vehicle_New_Cost_Amount','Vehicle_Symbol','Vehicle_Number_Of_Drivers_Assigned','Vehicle_Usage','Vehicle_Miles_To_Work','Vehicle_Days_Per_Week_Driven','Vehicle_Annual_Miles','Vehicle_Anti_Theft_Device','Vehicle_Passive_Restraint','Vehicle_Age_In_Years','Vehicle_Med_Pay_Limit','Vehicle_Bodily_Injury_Limit','Vehicle_Physical_Damage_Limit','Vehicle_Comprehensive_Coverage_Indicator','Vehicle_Comprehensive_Coverage_Limit','Vehicle_Collision_Coverage_Indicator','Vehicle_Collision_Coverage_Deductible','Driver_Total','Driver_Total_Male','Driver_Total_Female','Driver_Total_Single','Driver_Total_Married','Driver_Total_Related_To_Insured_Self','Driver_Total_Related_To_Insured_Spouse','Driver_Total_Related_To_Insured_Child','Driver_Total_Licensed_In_State','Driver_Minimum_Age','Driver_Maximum_Age','Driver_Total_Teenager_Age_15_19','Driver_Total_College_Ages_20_23','Driver_Total_Young_Adult_Ages_24_29','Driver_Total_Low_Middle_Adult_Ages_30_39','Driver_Total_Middle_Adult_Ages_40_49','Driver_Total_Adult_Ages_50_64','Driver_Total_Senior_Ages_65_69','Driver_Total_Upper_Senior_Ages_70_plus','Vehicle_Youthful_Driver_Indicator','Vehicle_Youthful_Driver_Training_Code','Vehicle_Youthful_Good_Student_Code','Vehicle_Driver_Points','Vehicle_Safe_Driver_Discount_Indicator','EEA_Liability_Coverage_Only_Indicator','EEA_Multi_Auto_Policies_Indicator','EEA_Policy_Zip_Code_3','EEA_Policy_Tenure','EEA_Agency_Type','EEA_Packaged_Policy_Indicator','EEA_Full_Coverage_Indicator','EEA_Prior_Bodily_Injury_Limit','EEA_PolicyYear','SYS_Renewed','SYS_New_Business','Annual_Premium','Claim_Count','Loss_Amount','Frequency','Severity','Loss_Ratio']
    lis=list(range(1,421))
    st=[str(i) for i in lis]
    j=0
    for i in range(len(claimsub)):
        csv_file = ("Training "+ st[j]+".csv")
        (pd.DataFrame.from_dict(data=claimsub[j], orient='index')
           .to_csv(csv_file, header=True))
        j+=1
    print('--------------CSV files created--------------------')

